# Prédire si un député va voter ou non 
### Utilisation d'une Random Forest avec comme target la variable 'Présence' (ie. député présent au vote ou non), les features utilisées seront des données sur le profil des députés

On choisit de découper la base selon les grands thèmes des scrutins, on pourra ainsi prédire si pour un thème donné les députés votent ou non (une random forest par thème)
On s'entraine donc sur un thème et on choisira quelques thèmes à étudier à la fin

## Création de la variable 'Présence' pour chaque député sur chaque scrutin (base entière)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

df_votants = pd.read_csv('database_deputes.csv', index_col=0)
df_votes2 = pd.read_csv('database_votes2.csv', index_col=0)

In [3]:
df_votes = df_votes2[['idScrutin', 'idVotant']]

In [4]:
copy = df_votes2.copy()
copy = copy.pivot(index = 'idVotant', columns = 'idScrutin', values = 'vote')

In [5]:
#on remplace les NaN par de l'abstention (en confondant abstention et non-votant)
copy = copy.fillna('0')

copy = copy.replace(['Pour'],'1')
copy = copy.replace(['Contre'],'1')
copy = copy.replace(['Non-votant'],'1')

In [6]:
copy2=copy.unstack()

In [7]:
copy2 = copy2.reset_index()
copy2 = copy2.rename(columns = {0 : 'présence'})

In [8]:
copy2 = copy2.sort_values(['idScrutin', 'idVotant'])
copy2.to_csv('présence.csv')


#### On merge la variable Présence sur la base complète

In [9]:
bjr = df_votes2.merge(copy2, on = ['idScrutin', 'idVotant'], how = 'outer')

In [10]:
df_indices = pd.read_csv('indices_2018.csv')

In [11]:
df_complet = df_indices.merge(bjr[['idScrutin','idVotant','présence']], on='idVotant', how ='outer')

In [12]:
df_scrutins2 = df_votes2.drop_duplicates('idScrutin')[['idScrutin','titre','demandeur','date_scrutin']]

In [13]:
df_complet=df_complet.merge(df_scrutins2, on = 'idScrutin', how='outer')

In [14]:
df_cluster = pd.read_csv('df_randomforest.csv')

C:\Users\Clara\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (25,26,27,28,40,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
df_cluster[df_cluster['cluster']==29].intitule

133                ['lutter', 'haine', 'internet']
159                ['lutter', 'haine', 'internet']
169       ['dumas', 'lutter', 'haine', 'internet']
183                ['lutter', 'haine', 'internet']
564                  ['fur', 'lutter', 'internet']
                            ...                   
289479             ['lutter', 'haine', 'internet']
289598            ['lutter', 'texte', 'paritaire']
289720             ['lutter', 'haine', 'internet']
289889             ['lutter', 'haine', 'internet']
290172                      ['lutter', 'deuxieme']
Name: intitule, Length: 1958, dtype: object

In [16]:
df_cluster=df_cluster[['idScrutin','cluster']]

In [17]:
df_cluster=df_cluster.drop_duplicates(['idScrutin','cluster'])

In [18]:
df_complet = df_complet.merge(df_cluster, on =['idScrutin'], how='outer')

####  On a le df complet avec les infos nécessaires sur les votants (id, pour, contre, abstention, accord, région, age, genre, parti) et sur les scrutins (thème, date) ainsi que la variable présence

In [19]:
#df_complet.to_csv('df_rf.csv')

#### On ne garde maintenant que les scrutins proposés après le 01-01-2020 afin de prédire sur cette période 

In [20]:
df_prediction = df_complet[df_complet['date_scrutin']>"2019-01-01"]

In [21]:
df_prediction = df_prediction[['idVotant','présence','Contre','Pour','indice_accord','taux_abstention','Région_x','age','Genre','CSP','Groupe politique (complet)','idScrutin','date_scrutin','cluster']]

### Nous choisissons sur quel cluster (ie, thème) travailler en ne gardant que les scrutins correspondant à ce thème

In [22]:
df_randomforest_19=df_prediction[df_prediction['cluster']==29]


In [23]:
df_randomforest_19_présence=df_randomforest_19['présence'].to_frame()

#### Features finales : 

In [24]:
df_randomforest_19.columns

Index(['idVotant', 'présence', 'Contre', 'Pour', 'indice_accord',
       'taux_abstention', 'Région_x', 'age', 'Genre', 'CSP',
       'Groupe politique (complet)', 'idScrutin', 'date_scrutin', 'cluster'],
      dtype='object')

#### Séparation entre features et target :

In [25]:
df_randomforest_19_présence=df_randomforest_19_présence.astype(int)
df_randomforest_19=df_randomforest_19.drop(['présence','idVotant','date_scrutin','idScrutin'], axis =1)
#df_randomforest_19=df_randomforest_19.drop('Groupe politique (complet)', axis=1)


In [26]:
df_randomforest_19_dummies=pd.get_dummies(df_randomforest_19)


In [42]:
label_19 = df_randomforest_19_présence['présence']


### Train/test

In [28]:
from sklearn.model_selection import train_test_split


In [29]:
train_features, test_features, train_labels, test_labels = train_test_split(df_randomforest_19_dummies, label_19, test_size = 0.25)

### Les données de la variable Présence sont déséquilibrées (plus de 0 que de 1), on les réequilibre

In [30]:
from sklearn.utils import resample,shuffle
test_label_counts = test_labels.value_counts()
test_features_absent = test_features[test_labels==0]
test_labels_absent = test_labels[test_labels==0]
features_test_less, labels_test_less = resample(test_features_absent,test_labels_absent,n_samples=test_label_counts[1],replace=False)
features_test_ = pd.concat([features_test_less,test_features[test_labels==1]])
labels_test_ = pd.concat([labels_test_less,test_labels[test_labels==1]])
test_features_, test_labels_ = shuffle(features_test_,labels_test_)
print(test_labels_.value_counts())

1    365
0    365
Name: présence, dtype: int64


In [31]:
train_label_counts = train_labels.value_counts()
train_features_absent = (train_features[train_labels==0])
train_labels_absent = (train_labels[train_labels==0])
features_train_less, labels_train_less = resample(train_features_absent,train_labels_absent,n_samples=train_label_counts[1],replace=False)
features_train_ = pd.concat([features_train_less,train_features[train_labels==1]])
labels_train_ = pd.concat([labels_train_less,train_labels[train_labels==1]])
train_features_, train_labels_ = shuffle(features_train_,labels_train_)
print(train_labels_.value_counts())

1    1227
0    1227
Name: présence, dtype: int64


### On fait tourner la RF

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 500 , max_depth=10)

rf.fit(train_features_, train_labels_);

In [33]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth':[5,6,7,8,10], 'n_estimators' : [100,200,300,500,1000]}
predictor= GridSearchCV(RandomForestClassifier(random_state=0),param_grid=param_grid)
predictor.fit(train_features_,train_labels_)
print('Paramètre sélectionné:',predictor.best_params_)
print('Score d\'apprentissage: ',predictor.score(train_features_,train_labels_))
print('Score de test: ',predictor.score(test_features_,test_labels_))

Paramètre sélectionné: {'max_depth': 8, 'n_estimators': 1000}
Score d'apprentissage:  0.6854115729421353
Score de test:  0.6328767123287671


In [34]:
predictions_test = rf.predict(test_features_)

In [36]:
print('Score d\'apprentissage: ',rf.score(train_features_,train_labels_))
print('Score de test: ',rf.score(test_features_,test_labels_))


Score d'apprentissage:  0.7033414832925835
Score de test:  0.6205479452054794


print("Feature ranking:")
for i, data_class in enumerate(df_randomforest_19_dummies.columns):
    print("{}. {} ({})".format(i + 1, data_class, importances[i]))


### Scores

Calcul du recall pour Random Forest

recall = recall_score(test_labels_, predictions_test, average='macro')
print('Recall: %.3f' % recall)



Calcul du F1-Score pour Random Forest

f1 = f1_score(test_labels_, predictions_test, average='macro')
print('F1-Score: %.3f' % f1)


## On fait tourner en enlevant une variable a chaque fois pour voir la différence avec l'accuracy normale (ie. avec toutes les variables)

In [41]:
for i in df_randomforest_19.columns:
    print(i)
    df_test = df_randomforest_19.drop([i], axis=1)
    df_test_dummies = pd.get_dummies(df_test)
    train_features, test_features, train_labels, test_labels = train_test_split(df_test_dummies, label_19, test_size = 0.25)
    from sklearn.utils import resample,shuffle
    test_label_counts = test_labels.value_counts()
    test_features_absent = test_features[test_labels==0]
    test_labels_absent = test_labels[test_labels==0]
    features_test_less, labels_test_less = resample(test_features_absent,test_labels_absent,n_samples=test_label_counts[1],replace=False)
    features_test_ = pd.concat([features_test_less,test_features[test_labels==1]])
    labels_test_ = pd.concat([labels_test_less,test_labels[test_labels==1]])
    test_features_, test_labels_ = shuffle(features_test_,labels_test_)
    #print(test_labels_.value_counts())
    train_label_counts = train_labels.value_counts()
    train_features_absent = (train_features[train_labels==0])
    train_labels_absent = (train_labels[train_labels==0])
    features_train_less, labels_train_less = resample(train_features_absent,train_labels_absent,n_samples=train_label_counts[1],replace=False)
    features_train_ = pd.concat([features_train_less,train_features[train_labels==1]])
    labels_train_ = pd.concat([labels_train_less,train_labels[train_labels==1]])
    train_features_, train_labels_ = shuffle(features_train_,labels_train_)
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 500 , max_depth=10)
    rf.fit(train_features_, train_labels_);
    predictions_test = rf.predict(test_features_)
    #print(train_labels_.value_counts())
    print('Score d\'apprentissage de: ',rf.score(train_features_,train_labels_))
    print('Score de test: ',rf.score(test_features_,test_labels_))
    

Contre
Score d'apprentissage de:  0.7067348678601876
Score de test:  0.6336515513126492
Pour
Score d'apprentissage de:  0.709717607973422
Score de test:  0.6237113402061856
indice_accord
Score d'apprentissage de:  0.694777397260274
Score de test:  0.6143867924528302
taux_abstention
Score d'apprentissage de:  0.6902581182348043
Score de test:  0.6035805626598465
Région_x
Score d'apprentissage de:  0.6989112227805695
Score de test:  0.6545226130653267
age
Score d'apprentissage de:  0.7172818791946308
Score de test:  0.63
Genre
Score d'apprentissage de:  0.7060280759702725
Score de test:  0.6299212598425197
CSP
Score d'apprentissage de:  0.6991666666666667
Score de test:  0.6288265306122449
Groupe politique (complet)
Score d'apprentissage de:  0.6909090909090909
Score de test:  0.6047120418848168
cluster
Score d'apprentissage de:  0.705661948376353
Score de test:  0.6214833759590793
